# Query DSL Lab

Remember that before launching the notebook you have to load the [opensearch settings file](../bash/opensearch):

```
source bash/opensearch
```

**You will have to edit the file and set the remote address of your opensearch instance.**

## Loading the datasets

In [ ]:
%%bash

cd ${DATASET_LOCATION}/movielens-latest-small

# movies
curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X DELETE \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/movies?pretty"
    
curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X PUT -H "Content-Type: application/json" \
    --data-binary @movies-index.json \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/movies"
    
curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X PUT -H "Content-Type: application/json" \
    --data-binary @movies-bulk.json \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/_bulk" > /dev/null

In [ ]:
%%bash

cd ${DATASET_LOCATION}/movielens-latest-small

# movies-tuned
curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X DELETE \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/movies-tuned?pretty"
    
curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X PUT -H "Content-Type: application/json" \
    --data-binary @movies-tuned-index.json \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/movies-tuned"
    
curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X PUT -H "Content-Type: application/json" \
    --data-binary @movies-tuned-bulk.json \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/_bulk" > /dev/null

In [ ]:
%%bash

cd ${DATASET_LOCATION}/movielens-latest-small

# ratings
curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X DELETE \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/ratings?pretty"
    
curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X PUT -H "Content-Type: application/json" \
    --data-binary @ratings-index.json \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/ratings"
    
curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X PUT -H "Content-Type: application/json" \
    --data-binary @ratings-bulk.json \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/_bulk" > /dev/null

## Let's practice

### Total number of ratings
How many ratins do we have in the `ratings` index?

In [ ]:
%%bash

curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X GET \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/:index_name/:method_name?pretty"

### Star Trek films sorted by year
Using the `movies-tuned` index, Look for Star Trek films sorted by year (most recent first). Show the title and the year of the film.

In [ ]:
%%bash

curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X GET -H "Content-Type: application/json" \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/movies-tuned/_search?pretty" -d '
{
    "query" : {
        "..." : {
            "title": "..."
        }
    },
    "sort": [
        {"...": {"...": "..."}}
    ],
    "_source": ["...", "..."]
}'

### Star Trek films average rating
Using the `ratings` index, compute the average rating of the Star Trek films.

Show the title and the year of the film.

Notice that we have to use the `title.raw` field for sorting, if you try to use the `title` field you will get an error.

In [ ]:
%%bash

curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X GET -H "Content-Type: application/json" \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/ratings/_search?pretty" -d '
{
    "query" : {
        "match_phrase" : {
            "title": "star trek"
        }
    },
    "aggs": {
        "per_film": {
            "...": {
                "field": "title.raw"
            },
            "aggs": {
                "avg_rating": {
                    "avg": {
                        "field": "rating"
                    }
                }
            }
        }
    },
    "size": 0
}'

### Sci-Fi and Action films from 2000 to 2010 not Star Trek and preferably Superman or Batman movies
Using the `movies-tuned` index, look for Sci-Fi and Action films from 2000 to 2010 that are not Star Trek movies and that preferably are Superman or Batman movies.

In [ ]:
%%bash

curl --silent --insecure -u ${OPENSEARCH_USER}:${OPENSEARCH_PASSWD} \
    -X GET -H "Content-Type: application/json" \
    "https://${OPENSEARCH_HOST}:${OPENSEARCH_PORT}/movies-tuned/_search?pretty" -d '
{
    "query" : {
        "bool": {
            "must_not": [
                {"...": {"title": "..."}}
            ],
            "should": [
                {"match": {"title": "..."}},
                {"match": {"title": "..."}}
            ],
            "filter": [
                {"range": {"...": {"from": ..., "to": ...}}},
                {"terms": {"genres": ["...", "..."]}}
            ]            
        }
    }
}'